## <font color=darkcyan> MCMC algorithms</font>
$
\newcommand{\PP}{\mathbb P}
\newcommand{\PE}{\mathbb E}
\newcommand{\Xset}{\mathsf{X}}
\newcommand{\nset}{\mathbb{N}}
\newcommand{\invcdf}[1]{F_{#1}^{\leftarrow}}
\newcommand{\rmd}{\mathrm{d}}
\newcommand{\rme}{\mathrm{e}}
$
In this notebook, we illustrate some results of the course **Simulation methods for generative models**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import scipy.stats as stats
from scipy.stats import expon, geom, norm
from math import pi
from pylab import *
%matplotlib inline

#### <font color=darkorange> The invariant measure of a Markov chain </font>



#### First example (see Exercise 7.4)



Consider a Gaussian AR($1$) process, $X_t= \mu + \phi X_{t-1} + \sigma Z_t$, where $(Z_t)_{t \in \nset}$ is an iid sequence of standard Gaussian random variables, independent of $X_0$. Assume that $|\phi| < 1$. Then, the Gaussian distribution with mean $\mu/(1-\phi)$ and variance $\sigma^2/(1-\phi^2)$ is a stationary distribution of the Markov chain. We check this property with an histogram of the values taken by a single trajectory of the Markov chain. 



In [ ]:
p,mu,phi,sig=10000,1,0.9,1
mc=npr.rand(1)*np.ones(p)
f=lambda x,m,sq: np.exp(-(x-m)**2/(2*sq))/np.sqrt(2*pi*sq)
mc[0]=0
for i in range(p-1):
    mc[i+1]=mu+phi*mc[i]+sig*npr.randn()
x=np.linspace(min(mc),max(mc),30)
plt.hist(mc,bins=80,density=True,edgecolor="black")
plt.plot(x,f(x,mu/(1-phi),sig**2/(1-phi**2)),color="red")
plt.title("Histogram of a trajectory of the MC. n="+str(p))
plt.show()



#### Second example (Exercise 7.5)

We here consider an example of a Markov chain (given during Tutorial 1) whose state space $\Xset= (0,1)$ is the open unit interval.
If the chain is at $x$, it picks one of the two intervals $(0,\ x)$ or $(x,\ 1)$ with equal probability $1/2$, and then moves to a point $y$ which is uniformly distributed in the chosen interval. The invariant distribution of the Markov chain has the cdf $x \mapsto \frac2\pi\, \mathrm{arcsin}( \sqrt{x} )$. By differentiation, we can get the associated density: $x \mapsto \frac{1}{\pi \sqrt{x(1-x)}}$ for all $x\in (0,1)$. We now check this property with an histogram of the values taken by the Markov chain. 


In [ ]:
m,p=500,10000
mc=npr.rand(1)*np.ones(p);
f=lambda x: 1/(pi*sqrt(x*(1-x))); 
x=np.arange(1,m)/m

for i in range(p-1):
    [a,b]=npr.rand(2)
    mc[i+1]=(a<0.5)*b*mc[i]+(a>0.5)*(mc[i]+b*(1-mc[i]))
plt.hist(mc,bins=40,density=True,edgecolor="black")
plt.plot(x,f(x),color="red")
plt.title("Histogram of a trajectory of the MC. n="+str(p))
plt.show()

We can also illustrate how the histogram converges to the density of the stationary distribution. This can be done with a dynamical animation with the module "animation" from matplotlib. 

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

m,p=500,10001
mc=npr.rand(1)*np.ones(p)
f=lambda x: 1/(pi*sqrt(x*(1-x)))
x=np.arange(1,m)/m

data = []
for i in range(p-1):
    [a,b]=npr.rand(2)
    mc[i+1]=(a<0.5)*b*mc[i]+(a>0.5)*(mc[i]+b*(1-mc[i]))
    
    if ((i+1)%100==0):
        data.append(list(mc[0:i+1]))


number_of_frames =100
fig, ax = plt.subplots()
plt.close()

def update_hist(num,data):
    ax.cla()
    ax.hist(data[num],bins = 40, density = True, edgecolor = "black")
    ax.plot(x,f(x),color="red")
    ax.set_title("Histogram of a trajectory of the MC. sample nb="+str((num+1)*100))
    
anim = animation.FuncAnimation(fig, update_hist, number_of_frames, fargs=(data, ),repeat = False)
rc('animation', html='jshtml')
anim

We now illustrate the law of large numbers with an example. Take $g(x)=\pi \sqrt{x(1-x)}$. Then, we expect that $\PP-a.s.$, 
$$
\lim_{n \to \infty}n^{-1}\sum_{i=1}^n g(X_i)=\pi(g)=\int_0^1 g(x)  \frac{1}{\pi \sqrt{x(1-x)}}\rmd x=\int_0^1 \rmd x=1
$$

In [ ]:
p=50000
mc=npr.rand(1)*np.ones(p)
g=lambda x: (pi*sqrt(x*(1-x)))
x=np.arange(1,p+1)/(p)
for i in range(p-1):
    [a,b]=npr.rand(2)
    mc[i+1]=(a<0.5)*b*mc[i]+(a>0.5)*(mc[i]+b*(1-mc[i]))
moy=np.cumsum(g(mc))/np.arange(1,p+1)
plot(x,moy)
plt.ylim([0.6,1.1])
axhline(y=1, color='red')
labels=["empirical means","theoretical value"]
legend(labels)
show()

#### <font color=darkorange> Symmetric Random Walk Metropolis Hasting algorithm </font>

We now consider a target distribution which is the mixture of two Gaussian distributions, one centered at $a$ and the other one centered at $-a$ 
$$
\pi(x)=\frac{1}{2}\left(\phi(x-a)+\phi(x+a)\right)=\frac{1}{2} \frac{\rme^{-(x-a)^2}}{\sqrt{2\pi}}+\frac{1}{2} \frac{\rme^{-(x+a)^2}}{\sqrt{2\pi}}
$$
where $\phi$ is the density of the centered standard normal distribution. 

To target this distribution, we sample according to a Symmetric Random Walk Metropolis Hasting algorithm. When the chain is at the state $X_k$, we propose a candidate $Y_{k+1}$ according to $Y_{k+1}=X_k+Z_k$ where $Z_k\sim {\mathcal N}(0,1)$ and then we accept $X_{k+1}=Y_{k+1}$ with probability $\alpha(X_k,Y_{k+1})$, where $\alpha(x,y)=\frac{\pi(y)}{\pi(x)} \wedge 1$. Otherwise, $X_{k+1}=X_{k}$. 

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

p,a,r=20001,3,200
mc=npr.randn()*np.ones(p)
cible=lambda x,a: (np.exp(-(x-a)**2/2)+np.exp(-(x+a)**2/2))/np.sqrt(8*pi)

data=[]
for i in range(p-1):
    v=npr.randn()
    alpha=cible(mc[i]+v,a)/cible(mc[i],a)
    mc[i+1]=mc[i]
    if (npr.rand()<alpha): 
        mc[i+1] += v        
    if ((i+1)%r==0):
        data.append(list(mc[0:i]))

s=np.cumsum(mc)/np.arange(1,p+1)      
x=np.linspace(min(mc),max(mc),100)
number_of_frames=int(p/r)
fig, ax = plt.subplots(3,1, figsize=(15,9))
plt.close()

def update_hist(num,data):
    fig.tight_layout(rect = [0, 0, 1, 0.9]) 
    ax[0].cla()
    ax[0].set_ylim(0, 0.25)
    ax[0].hist(data[num],bins = 40, density = True, edgecolor = "black")
    ax[0].plot(x,cible(x,a),color="red")
    ax[0].plot(mc[num*r],0.005,"o",color="yellow")
    ax[0].set_title("Histogram of a trajectory of the MC. sample nb="+str(r*(num+1)))
    ax[1].cla()
    ax[1].set_xlim(0,p)
    ax[1].set_ylim(-6,6)
    ax[1].plot(mc[0:num*r])
    ax[1].set_title("Trajectory of the MC. sample nb="+str(r*(num+1)))
    ax[2].cla()
    ax[2].set_xlim(0,p)
    ax[2].set_ylim(-6,6)
    ax[2].plot(s[0:num*r])
    ax[2].axhline(y=1, color='red')
    ax[2].set_title("Empirical mean. sample nb="+str(r*(num+1)))

    
anim = animation.FuncAnimation(fig, update_hist, number_of_frames, fargs=(data, ),repeat = False)
rc('animation', html='jshtml')
anim

#### <font color=darkorange> Independent Metropolis Hasting algorithm </font>

We again consider a target distribution which is a mixture of two Gaussian distributions, one centered at $a$ and the other one centered at $-a$ 
$$
\pi(x)=\frac{1}{2}\left(\phi(x-a)+\phi(x+a)\right)=\frac{1}{2} \frac{\rme^{-(x-a)^2}}{\sqrt{2\pi}}+\frac{1}{2} \frac{\rme^{-(x+a)^2}}{\sqrt{2\pi}},
$$
where $\phi$ is the density of the centered standard normal distribution. 

To target this distribution, we sample according to a Metropolis Hasting algorithm with independent proposal. When the chain is at the state $X_k$, we propose a candidate $Y_{k+1}$ according to $Y_{k+1}=Z_k$ where $Z_k\sim {\mathcal N}(\theta,\sigma^2)$ and then we accept $X_{k+1}=Y_{k+1}$ with probability $\alpha(X_k,Y_{k+1})$, where $\alpha(x,y)=\frac{\pi(y)q(x)}{\pi(x)q(y)} \wedge 1=\frac{\pi(y)/q(y)}{\pi(x)/q(x)} \wedge 1$ and $q$ is the density of ${\mathcal N}(\theta,\sigma^2)$. Otherwise, $X_{k+1}=X_{k}$. 

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

counts = 0
p,a,r,theta,sigma=20001,3,200,2,-4
mc=npr.randn()*np.ones(p)
cible=lambda x,a: (np.exp(-(x-a)**2/2)+np.exp(-(x+a)**2/2))/np.sqrt(8*pi)
densi=lambda x,m,s: np.exp(-(x-m)**2/(2*s**2))/np.sqrt(2*pi*s**2)

data=[]
for i in range(p-1):
    v=sigma*npr.randn()+theta
    alpha=cible(v,a)*densi(mc[i],theta,sigma)/(cible(mc[i],a)*densi(v,theta,sigma))
    mc[i+1]=mc[i]
    if v>6:
      counts += 1
    if (npr.rand()<alpha): 
        mc[i+1] = v        
    if ((i+1)%r==0):
        data.append(list(mc[0:i]))

x=np.linspace(min(mc),max(mc),100)
maxi=max(cible(x,a)/densi(x,theta,sigma))
number_of_frames=int(p/r)
fig, ax = plt.subplots(2,1, figsize=(15,9))
plt.close()


def update_hist(num,data):
    fig.tight_layout(rect = [0, 0, 1, 0.9]) 
    ax[0].cla()
    ax[0].plot(mc[num*r],0.005,"o",color="yellow")
    ax[0].set_ylim(0, 0.25)
    ax[0].hist(data[num],bins = 40, density = True, edgecolor = "black")
    ax[0].plot(x,cible(x,a),color="red")
    ax[0].plot(x,densi(x,theta,sigma),color="orange")
    ax[0].plot(x,cible(x,a)/densi(x,theta,sigma)*(0.2/maxi),color="cyan")
    labels=["MC","target","propos.","target/propos."]
    ax[0].legend(labels)
    ax[0].set_title("Histogram of a trajectory of the MC. sample nb="+str(r*(num+1)))
    ax[1].cla()
    ax[1].set_xlim(0,p)
    ax[1].set_ylim(-6,6)
    ax[1].plot(mc[0:num*r], color="blue")
    ax[1].set_title("Trajectory of the MC. sample nb="+str(r*(num+1)))
    
    
anim = animation.FuncAnimation(fig, update_hist, number_of_frames, fargs=(data, ),repeat = False)
rc('animation', html='jshtml')
anim

